In [1]:
!pip install spacy

In [2]:
!wget http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz

--2020-01-04 16:46:15--  http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.20
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.20|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3127238 (3.0M) [application/x-gzip]
Saving to: ‘review_polarity.tar.gz’

review_polarity.tar 100%[===================>]   2.98M  3.79MB/s    in 0.8s    

2020-01-04 16:46:16 (3.79 MB/s) - ‘review_polarity.tar.gz’ saved [3127238/3127238]



In [4]:
import tarfile
tf = tarfile.open("review_polarity.tar.gz")
tf.extractall()
print('Done!')

Done!


In [21]:
import numpy as np
import re
import nltk
from sklearn.datasets import load_files
import pickle
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
%cd /content

/content


In [0]:
movie_data = load_files('/content/txt_sentoken')
X, y = movie_data.data, movie_data.target

In [0]:
documents = []
from nltk.stem import WordNetLemmatizer
stemmer = WordNetLemmatizer()

for sen in range(0, len(X)):
    #removing special characters
    document = re.sub(r'\W', ' ', str(X[sen]))
    
    #removing all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    #removing all single characters from start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)
    
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    document = re.sub(r'^b\s+', ' ', document)
    
    document = document.lower()
    
    document = document.split()
    
    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7,
                            stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(documents).toarray()

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidvectorizer = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7, 
                                stop_words=stopwords.words('english'))
X = tfidvectorizer.fit_transform(documents).toarray()

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [26]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [0]:
y_pred = classifier.predict(X_test)

In [29]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print('Confusion Matrix: ' + confusion_matrix(y_test, y_pred))
print('Classification Matrix: ' + classification_report(y_test, y_pred))
print('Accuracy Matrix: ' + accuracy_score(y_test, y_pred))

UFuncTypeError: ignored